In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd # remove when finished
from implementations import *
from tools import *
from helpers import *

In [2]:
data_tr = pd.read_csv('data/train.csv')
idx_tr = data_tr.Id

In [3]:
features_names = data_tr.columns.drop(['Id','Prediction'])
# features_names

In [4]:
data_tr.describe()

,Id,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
count,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,...,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000
mean,224999.500000,-49.023079,49.239819,81.181982,57.895962,-708.420675,-601.237051,-709.356603,2.373100,18.917332,...,-0.010119,209.797178,0.979176,-348.329567,-399.254314,-399.259788,-692.381204,-709.121609,-709.118631,73.064591
std,72168.927986,406.345647,35.344886,40.828691,63.655682,454.480565,657.972302,453.019877,0.782911,22.273494,...,1.812223,126.499506,0.977426,532.962789,489.338286,489.333883,479.875496,453.384624,453.389017,98.015662
min,100000.000000,-999.000000,0.000000,6.329000,0.000000,-999.000000,-999.000000,-999.000000,0.208000,0.000000,...,-3.142000,13.678000,0.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,0.000000
25%,162499.750000,78.100750,19.241000,59.388750,14.068750,-999.000000,-999.000000,-999.000000,1.810000,2.841000,...,-1.575000,123.017500,0.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,0.000000
50%,224999.500000,105.012000,46.524000,73.752000,38.467500,-999.000000,-999.000000,-999.000000,2.491500,12.315500,...,-0.024000,179.739000,1.000000,38.960000,-1.872000,-2.093000,-999.000000,-999.000000,-999.000000,40.512500
75%,287499.250000,130.606250,73.598000,92.259000,79.169000,0.490000,83.446000,-4.593000,2.961000,27.591000,...,1.561000,263.379250,2.000000,75.349000,0.433000,0.503000,33.703000,-2.457000,-2.275000,109.933750
max,349999.000000,1192.026000,690.075000,1349.351000,2834.999000,8.503000,4974.979000,16.690000,5.684000,2834.999000,...,3.142000,2003.976000,3.000000,1120.573000,4.499000,3.141000,721.456000,4.500000,3.142000,1633.433000


In [5]:
def missing_rate(data):
    features = data.columns.drop(['Id','Prediction'])
    df_ = pd.DataFrame({'idx':[np.nan]})
    for feature in features:
        data2 = data.loc[(data[feature]==-999)].copy()
        missing_rate = data2[feature].size/data[feature].size
        df1 = pd.DataFrame({feature:[missing_rate]})
        df_ = pd.concat([df_,df1],axis=1)
    df_ = df_.drop(columns='idx')
    df_ = df_.T.rename(columns={0:'rate'})
    return df_

In [6]:
def drop_columns(df_miss):
    names_70 = list(df_miss.loc[df_miss.rate==0.709828].index)
    names_30 = list(df_miss.loc[df_miss.rate==0.399652].index)
    names_phi = list(['PRI_jet_leading_phi','PRI_jet_subleading_phi','PRI_lep_phi','PRI_met_phi','PRI_tau_phi','PRI_jet_num'])
    corr0 = list(['DER_mass_MMC','DER_pt_tot','PRI_tau_pt','PRI_lep_pt'])
    corr1_2 = list(['DER_mass_MMC','DER_pt_h', 'DER_sum_pt', 'PRI_met_sumet','PRI_jet_all_pt','PRI_lep_pt'])
    df_drop_name = {'names_70':names_70,'names_30':names_30,'names_phi':names_phi,'corr0':corr0,'corr1_2':corr1_2}
    
    return df_drop_name

In [7]:
df_miss_tr = missing_rate(data_tr)
df_miss_tr

,rate
DER_mass_MMC,0.152456
DER_mass_transverse_met_lep,0.000000
DER_mass_vis,0.000000
DER_pt_h,0.000000
DER_deltaeta_jet_jet,0.709828
DER_mass_jet_jet,0.709828
DER_prodeta_jet_jet,0.709828
DER_deltar_tau_lep,0.000000
DER_pt_tot,0.000000
DER_sum_pt,0.000000


In [8]:
# replacing the -999 with mode in signal or background

DER_mode_s = data_tr.loc[(data_tr['DER_mass_MMC']!=-999)].loc[(data_tr['Prediction'])=='s']['DER_mass_MMC'].value_counts().index[0]#mean(axis=0)
DER_mode_b = data_tr.loc[(data_tr['DER_mass_MMC']!=-999)].loc[(data_tr['Prediction'])=='b']['DER_mass_MMC'].value_counts().index[0]#value_counts().index[0]

idx_s = data_tr.loc[(data_tr['DER_mass_MMC']==-999)].loc[(data_tr['Prediction'])=='s']['DER_mass_MMC'].index
idx_b = data_tr.loc[(data_tr['DER_mass_MMC']==-999)].loc[(data_tr['Prediction'])=='b']['DER_mass_MMC'].index

data_tr.loc[idx_s,'DER_mass_MMC']=DER_mode_s
data_tr.loc[idx_b,'DER_mass_MMC']=DER_mode_b


In [9]:
drop_col = drop_columns(df_miss_tr)
drop_col
# ['DER_mass_MMC']['DER_pt_tot'][('DER_sum_pt', )'PRI_tau_pt', 'PRI_lep_pt']
# 

# ['DER_mass_MMC']['DER_pt_h', 'DER_sum_pt', 'PRI_met_sumet', ('PRI_jet_leading_pt'),'PRI_jet_all_pt']
# ['DER_pt_ratio_lep_tau', 'PRI_lep_pt']

# ['DER_mass_MMC']['DER_pt_h', 'DER_sum_pt', 'PRI_met_sumet', 'PRI_jet_leading_pt','PRI_jet_all_pt']
# [('DER_pt_ratio_lep_tau'), 'PRI_lep_pt']

# PRI_jet_leading_pt DER_sum_pt

# list1 = list(['DER_mass_MMC', 'DER_mass_vis','DER_pt_h', 'DER_pt_tot','DER_sum_pt','PRI_tau_pt','PRI_lep_pt'])

# list2 = list(['DER_mass_MMC', 'DER_mass_vis','DER_pt_h', 'DER_sum_pt', 'PRI_met_sumet', 'PRI_jet_leading_pt','PRI_jet_all_pt','DER_pt_ratio_lep_tau', 'PRI_lep_pt'])

# test = list(set(list1+list2))
# data_tr.loc[data_tr['Prediction']=='s'].describe()[test]-data_tr.loc[data_tr['Prediction']=='b'].describe()[test]
# ['DER_mass_MMC', 'DER_mass_vis']['DER_pt_h', 'DER_sum_pt', 'PRI_met_sumet', 'PRI_jet_leading_pt','PRI_jet_all_pt']
# ['DER_pt_ratio_lep_tau', 'PRI_lep_pt']


{'names_70': ['DER_deltaeta_jet_jet',
  'DER_mass_jet_jet',
  'DER_prodeta_jet_jet',
  'DER_lep_eta_centrality',
  'PRI_jet_subleading_pt',
  'PRI_jet_subleading_eta',
  'PRI_jet_subleading_phi'],
 'names_30': ['PRI_jet_leading_pt',
  'PRI_jet_leading_eta',
  'PRI_jet_leading_phi'],
 'names_phi': ['PRI_jet_leading_phi',
  'PRI_jet_subleading_phi',
  'PRI_lep_phi',
  'PRI_met_phi',
  'PRI_tau_phi',
  'PRI_jet_num'],
 'corr0': ['DER_mass_MMC', 'DER_pt_tot', 'PRI_tau_pt', 'PRI_lep_pt'],
 'corr1_2': ['DER_mass_MMC',
  'DER_pt_h',
  'DER_sum_pt',
  'PRI_met_sumet',
  'PRI_jet_all_pt',
  'PRI_lep_pt']}

'DER_deltaeta_jet_jet','DER_mass_jet_jet','DER_prodeta_jet_jet','DER_lep_eta_centrality','PRI_jet_subleading_pt','PRI_jet_subleading_eta','PRI_jet_subleading_phi': missing rate = 70.9% (PRI_jet_num <= 1)

'PRI_jet_leading_pt','PRI_jet_leading_eta','PRI_jet_leading_phi': missing rate = 39.9% (PRI_jet_num = 0)

In [10]:
def group_features_by_jet(df):
    return {  
        0: df.loc[df['PRI_jet_num'] == 0].copy(),
        1: df.loc[df['PRI_jet_num'] == 1].copy(),
        2: df.loc[(df['PRI_jet_num'] == 2) | (df['PRI_jet_num'] == 3)].copy(),
    }

In [11]:
feature_tr_grp = group_features_by_jet(data_tr)

In [12]:
feature_tr_grp[0] = feature_tr_grp[0].drop(columns = list(set(drop_col['names_phi']+drop_col['names_70']+drop_col['names_30']+drop_col['corr0'])))
feature_tr_grp[0] = feature_tr_grp[0].drop(columns = 'PRI_jet_all_pt')# all 0s
feature_tr_grp[1] = feature_tr_grp[1].drop(columns = list(set(drop_col['names_phi']+drop_col['names_70']+drop_col['corr1_2'])))
feature_tr_grp[2] = feature_tr_grp[2].drop(columns = list(set(drop_col['names_phi']+drop_col['corr1_2'])))

In [13]:
# ridge_regression using cv
w = []
acc_tr = []
acc_te = []
lambda_ = 0.0001
degrees = [15,15,14]
k_fold = 10
seed = 10

for i in range(len(feature_tr_grp)):
    x = feature_tr_grp[i].drop(columns = ['Id','Prediction']).values
    x = log_process(x)
    x = build_poly(x,degrees[i])
    x_tr,_,_ = standardize(x)
    y_tr = feature_tr_grp[i].Prediction.replace(['s','b'],[1,-1]).values
    w_tmp,acc_tr_tmp,acc_te_tmp = cv_loop(y_tr, x_tr,k_fold,seed,ridge_regression,lambda_=lambda_)
    w.append(w_tmp)
    acc_tr.append(acc_tr_tmp)
    acc_te.append(acc_te_tmp)

In [16]:
acc_tr

[0.8424879339872333, 0.798046998137269, 0.8220925423884575]

In [17]:
acc_te

[0.8419777800020016, 0.7967113747743101, 0.818775847808106]

In [29]:
# least square
w = []
acc_tr = []
acc_te = []
degrees = [15,15,14]
k_fold = 10
seed = 10

for i in range(len(feature_tr_grp)):
    x = feature_tr_grp[i].drop(columns = ['Id','Prediction']).values
    x = log_process(x)
    x = build_poly(x,degrees[i])
    x_tr,_,_ = standardize(x)
    y_tr = feature_tr_grp[i].Prediction.replace(['s','b'],[1,-1]).values
    w_tmp,acc_tr_tmp,acc_te_tmp = cv_loop(y_tr, x_tr,k_fold,seed, least_squares)
    w.append(w_tmp)
    acc_tr.append(acc_tr_tmp)
    acc_te.append(acc_te_tmp)

In [14]:
k_fold = 2
seed = 10
max_iters = 500
w = []
acc_tr = []
acc_te = []

#for i in range(len(feature_tr_grp)):
i=0
x = feature_tr_grp[i].drop(columns = ['Id','Prediction']).values
x = log_process(x)
x = build_polynomial_features(x,2)
x_tr,_,_ = standardize(x)
y_tr = feature_tr_grp[i].Prediction.replace(['s','b'],[1,-1]).values

loss,w_tmp = logistic_regression_GD(y_tr, x_tr, max_iters)
y_tr = y_tr.reshape(len(y_tr),1)
y_tr_pred = predict_labels(w_tmp, x_tr)
acc_tr_tmp = compute_accuracy(y_tr_pred,y_tr)
w.append(w_tmp)
acc_tr.append(acc_tr_tmp)   


In [23]:
acc_tr_tmp

array([0.83198383])